In [176]:
import yfinance as yf
import pandas as pd
from pathlib import Path

In [ ]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
backup.to_csv(result_folder / 'known_tickers.csv', index=False)

In [178]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'value_score', 
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'P/E (TTM)', 
    'ROA%',
    ])
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    try:
        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            format(2*(ticker.info['returnOnAssets'] * 100)/ticker.info['trailingPE'], '.3f'),
            format(analyst_mean, '.3f'), 
            format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.3f') if 'Shares' in ticker.insider_purchases.columns else 0,
            ticker.info['sector'], 
            ticker.info['industry'], 
            format(ticker.info['trailingPE'], '.3f'), 
            format(ticker.info['returnOnAssets'] * 100, '.3f'),
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks, target was {len(symbols)}')

,Ticker,short_name,value_score,analyst_mean%,insider_buy%,sector,industry,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,0.343,0.275,<NA>,Consumer Cyclical,Specialty Retail,22.342,3.835
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,0.255,0.334,<NA>,Technology,Information Technology Services,12.474,1.588
SAF.PA,SAF.PA,SAFRAN,0.145,0.002,<NA>,Industrials,Aerospace & Defense,66.128,4.785
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,0.724,0.100,<NA>,Industrials,Aerospace & Defense,19.017,6.888
RNMBY,RNMBY,Rheinmetall AG,0.115,0.000,<NA>,Industrials,Aerospace & Defense,103.617,5.966
DRS,DRS,"Leonardo DRS, Inc.",0.240,0.246,0.003,Industrials,Aerospace & Defense,37.619,4.519
HAGHY,HAGHY,HENSOLDT AG,0.011,0.000,<NA>,Industrials,Aerospace & Defense,503.571,2.786
SAABF,SAABF,Saab AB,0.159,0.000,<NA>,Industrials,Aerospace & Defense,47.548,3.783
BCKIY,BCKIY,Babcock International Group Plc,0.506,0.000,<NA>,Industrials,Engineering & Construction,20.083,5.078
EADSY,EADSY,Airbus SE,0.147,-0.010,<NA>,Industrials,Aerospace & Defense,32.879,2.421


Got data for 31 stocks, target was 31


In [179]:
print(yf.Ticker('RNMBY').info.keys())

dict_keys(['address1', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'tradeable', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'heldPercentInsiders', 'heldPer